# DWD Weather Station Data

In [1]:
import os
import requests
import bz2
import gzip
import pygrib
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from geopy.distance import geodesic

In [ ]:
path = ''

df = pd.read_csv(path)

In [ ]:
df